### Análisis de Impacto Financiero -
En este notebook se utiliza el modelo #3 (calculo de metricas con flota combinada) con el objetivo de evaluar el impacto incremental de cada vehiculo convencional incluido. Para ello es necesario repetir el análisis aumentando gradualmente la cantidad de vehículos convencionales y eléctricos. Luego, se obtiene el calculo de los promedios o tendencias más específicas para cada caso.

Se importan las librerias necesarias.

In [ ]:
import pandas as pd 
import numpy_financial as npf

Se importan los datasets que contienen la información requerida para el análisis.

In [ ]:
future_data = pd.read_csv(r"..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Output_Anualized.csv")
future_data = future_data[future_data['industry']=='FHV - High Volume']

future_data.rename(columns={
    'anual_income_per_vehicle': 'Income_per_Vehicle (USD)',
    'anual_distance_per_vehicle': 'Miles_per_Vehicle',
    'anual_total_co2_emission': 'Total_CO2_Emissions'
}, inplace=True)
future_data =  future_data[future_data['year']>=2025]

# Dataset de VE
df_autos = pd.read_csv(r'..\datasets\2. Depurados\ElectricCarData_Clean.csv')
df_autos.rename(columns={
    'brand': 'Brand',
    'model': 'Model',
    'efficiency_whkm': 'Efficiency (Wh/km)'
}, inplace=True)

df_autos['Efficiency (kWh/mile)'] = df_autos['Efficiency (Wh/km)']/1000* 0.1688666667 
df_autos['Precio Dolar'] = df_autos['price_euro'] * 1.06

Se obtiene el promedio de las variables (ingreso anual y millas recorridas por año) para un primer acercamiento y entendimiento del comportamiento de las variables.

In [3]:
promedio_ingresos = round(future_data["Income_per_Vehicle (USD)"].mean(),2)
promedio_millas = round(future_data["Miles_per_Vehicle"].mean(),2)
print(f'El promedio de ingresos anuales es: {promedio_ingresos}')
print(f'El promedio de millas recorridas anualmente es : {promedio_millas}')

El promedio de ingresos anuales es: 59327.19
El promedio de millas recorridas anualmente es : 10873.13


A continuación se asigna a cada dato una variable que almacena su información.

In [4]:
año = 2025
for _, row in future_data.iterrows():  # iterrows() en lugar de interrows()
    globals()[f'Ingresos_año_{año}'] = round(row["Income_per_Vehicle (USD)"],2)
    print(f'Los ingresos para el año {año} son {globals()[f"Ingresos_año_{año}"]}')
    año += 1

Los ingresos para el año 2025 son 65357.06
Los ingresos para el año 2026 son 64802.76
Los ingresos para el año 2027 son 64302.32
Los ingresos para el año 2028 son 63485.52
Los ingresos para el año 2029 son 63408.43
Los ingresos para el año 2030 son 63019.9
Los ingresos para el año 2031 son 62664.8
Los ingresos para el año 2032 son 61984.98
Los ingresos para el año 2033 son 62013.27
Los ingresos para el año 2034 son 61726.06
Los ingresos para el año 2035 son 19833.94


In [5]:
ingresos_años = []
for _, row in future_data.iterrows():
    ingresos_años.append(round(row["Income_per_Vehicle (USD)"],2))
print(ingresos_años)

[65357.06, 64802.76, 64302.32, 63485.52, 63408.43, 63019.9, 62664.8, 61984.98, 62013.27, 61726.06, 19833.94]


In [6]:
año = 2025
for _, row in future_data.iterrows():
    globals()[f'Millas_Año_{año}'] = round(row["Miles_per_Vehicle"],2)
    print(f'Las Millas recorridas en el año {año} son {globals()[f"Millas_Año_{año}"]}')
    año+=1

Las Millas recorridas en el año 2025 son 11762.56
Las Millas recorridas en el año 2026 son 11703.94
Las Millas recorridas en el año 2027 son 11656.52
Las Millas recorridas en el año 2028 son 11556.92
Las Millas recorridas en el año 2029 son 11587.08
Las Millas recorridas en el año 2030 son 11563.32
Las Millas recorridas en el año 2031 son 11546.49
Las Millas recorridas en el año 2032 son 11474.36
Las Millas recorridas en el año 2033 son 11527.96
Las Millas recorridas en el año 2034 son 11525.51
Las Millas recorridas en el año 2035 son 3699.81


In [7]:
millas_años = []
for _, row in future_data.iterrows():
    millas_años.append(round(row["Miles_per_Vehicle"],2))
print(millas_años)

[11762.56, 11703.94, 11656.52, 11556.92, 11587.08, 11563.32, 11546.49, 11474.36, 11527.96, 11525.51, 3699.81]


### Clases:

Se crea una clase «Auto» que contiene caracteristicas generales de este objeto. Luego en cada clase que hereda (Auto Electrico y convencional) se personalizan estas propiedades y se agregan nuevas.

Tambien se crean las funciones para el calculo del Cash Flow y los demas indicadores financieros.

In [8]:
class Auto:
    def __init__(self,tasa_descuento, precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, 
                 ingresos_anuales):
        self.precio = precio
        self.iva = iva
        self.licencia_1 = licencia_1
        self.licencia_2 = licencia_2
        self.placa = placa
        self.inspeccion = inspeccion
        self.seguro = seguro
        self.recorrido_anual = recorrido_anual
        self.ingresos_anuales = ingresos_anuales
        self.tasa_descuento = tasa_descuento
    
    def inversion_inicial(self):
        raise NotImplementedError
    
    def costos_operativos(self):
        raise NotImplementedError
    
    def flujo_neto_anual(self):
        return self.ingresos_anuales - self.costos_operativos()
    
    def flujo_caja_proyectado(self, años=7):
        
        # Inversión inicial negativa porqué es el año 0
        inversion_inicial = -self.inversion_inicial()
        
        ingresos_año = [0]
        costos_año = [0]
        flujo_neto = [inversion_inicial]  # Año 0: 
        flujo_descuento = [inversion_inicial]  # Año 0 Inversión inicial negativa
        ingresos_descuento_año = [0]
        
        # Se calcula el flujo neto anual
        for año in range(1, años + 1):
            self.ingresos_anuales = ingresos_años[año-1]
            self.recorrido_anual = millas_años[año-1]
            flujo_neto_actual = self.flujo_neto_anual()
            flujo_neto.append(flujo_neto_actual)
            flujo_descuento_actual = flujo_neto_actual / (1 + self.tasa_descuento) ** año
            flujo_descuento.append(flujo_descuento_actual)
            ingresos_año.append(self.ingresos_anuales)
            ingresos_descuento_año.append(ingresos_años[año-1]  / (1 + self.tasa_descuento) ** año)
            costos_año.append(self.costos_operativos())
        
        # Se crea el DataFrame de resultados
        df_flujo_caja = pd.DataFrame({
            'Año': range(0, años + 1),
            'Ingresos': ingresos_año, 
            'Costos Operativos': costos_año,
            'Flujo Neto': flujo_neto,
            'Flujo Neto Descontado': flujo_descuento,
            'Ingresos Descontado': ingresos_descuento_año        
        })
        
        # Calculo el flujo total descontado para el ROI
        return df_flujo_caja

    def calcular_roi(self, flujo_total):
        roi = flujo_total/ self.inversion_inicial() *100
        roi_mensual_simple = roi/7/12
        #roi_mensual_compuesto = ((1+ flujo_total/ self.inversion_inicial())**(1/(7*12))-1)*100 #- self.inversion_inicial()
        return round(roi, 2),round(roi_mensual_simple, 2)#,round(roi_mensual_compuesto, 2)

    def calcular_ir(self, ingreso_total):
        ir = ingreso_total / abs(self.inversion_inicial())
        return round(ir, 2)

    def calcular_payback_period(self):
        flujo_acumulado = 0
        payback = 0
        for año in range(1, 8):
            flujo_anual = self.flujo_neto_anual()
            flujo_acumulado += flujo_anual
            if flujo_acumulado >= abs(self.inversion_inicial()):
                payback = año
                break
        return payback if payback > 0 else "No recuperado en 7 años"
    

Creación de clase de «Auto Convencional»

In [9]:
class AutoConvencional(Auto): #Licencia_1 = unico pago, licencia_2 = anual
    def __init__(self, tasa_descuento, precio=36200, iva=0.08875, licencia_1=550,licencia_2=100, placa = 300,
                 seguro=5000, inspeccion = 150,
                 eficiencia=25 , mantenimiento=0.15, recorrido_anual=promedio_millas, #eficiencia en MPG / mantenimiento = USD x Milla
                 ingresos_anuales=promedio_ingresos):
        super().__init__(tasa_descuento,precio, iva, licencia_1, licencia_2, placa, inspeccion,
                         seguro, recorrido_anual, ingresos_anuales) # Se usa el super constructor para evitar redundancia ya que es una clase hija
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
    
    def inversion_inicial(self):
        return self.precio * (1 + self.iva) + self.licencia_1
    
    def costos_operativos(self):
        precio_gasolina_galon = 3.06 #USD x galon
        costo_gasolina_anual = (self.recorrido_anual / self.eficiencia) * precio_gasolina_galon
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        costo_anual_salario = 2342 * 20  # Precio medio de la hora de un conductor de autos convencionales.
        return costo_gasolina_anual + costo_mantenimiento + self.seguro + self.licencia_2 + self.placa + self.inspeccion + costo_anual_salario

Creación de clase de «Auto Electrico»

In [10]:
class AutoElectrico(Auto): #Descuento de 5% al 10% en licencia ; Excencion del impuesto de venta
    def __init__(self,tasa_descuento, precio, eficiencia, iva=0, licencia_1=495,licencia_2=100, placa = 300,
                 seguro=1500, inspeccion = 100, mantenimiento=0.03, wallbox=600, 
                 recambio_bateria=5000, #mantenimiento = USD x Milla
                 recorrido_anual=promedio_millas, ingresos_anuales=promedio_ingresos): 
        super().__init__(tasa_descuento, precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, 
                         ingresos_anuales)
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
        self.wallbox = wallbox
        self.recambio_bateria = recambio_bateria
        self.tasa_descuento = tasa_descuento
        self.licencia_1=licencia_1
    
    def inversion_inicial(self):
        return self.precio + self.licencia_1 + self.wallbox
    
    def costos_operativos(self):
        precio_electricidad_kWh = 0.13 #USD x kHw
        costo_electricidad_anual = (self.recorrido_anual * self.eficiencia) * precio_electricidad_kWh
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        costo_anual_bateria = self.recambio_bateria / 12.5
        costo_anual_salario = 2342 * 22 # Promedio de horas de menejo de un conductor por año * Precio promedio de la hora de un chofer de taxis en NY para VE.
        return costo_electricidad_anual + costo_mantenimiento + costo_anual_bateria + self.seguro + self.licencia_2 + self.placa + self.inspeccion + costo_anual_salario

### Modelo #3:
Análisis Financiero de Flota Combinada (VE y auto convencional)

Esta función estudia el comportamiento de cada uno de los prototipos del Dataset de VE relacionado a los indicadores financieros anteriormente delineados.
En cuanto al auto convencional se construye un propotipo prototipo como muestra.

In [11]:
def calcular_metricas_flota(tasa_descuento, cantidad_ve, cantidad_conv):
    resultados = []
    flujo_neto_comb_lista = []
    for _, fila in df_autos.iterrows():
        # Se extraen los datos del Dataset
        marca = fila['Brand']
        modelo = fila['Model']
        precio_ve = fila['Precio Dolar']
        eficiencia_ve = fila['Efficiency (kWh/mile)']
        eficiencia_conv = 25

        eficiencia_total = (33.7 /  eficiencia_conv * cantidad_conv + eficiencia_ve * cantidad_ve) / (cantidad_ve+cantidad_conv)

        # Se instancian los autos segun su naturaleza (electrico o convencional)
        auto_electrico = AutoElectrico(tasa_descuento, precio=precio_ve, eficiencia=eficiencia_ve)
        auto_convencional = AutoConvencional(tasa_descuento)


        # Se calcula el flujo de caja proyectado para cada tipo de auto
        flujo_ve = auto_electrico.flujo_caja_proyectado()
        flujo_conv = auto_convencional.flujo_caja_proyectado()
        flujo_neto_comb_lista =  cantidad_ve * flujo_ve['Flujo Neto Descontado'] +  cantidad_conv  * flujo_conv['Flujo Neto Descontado']

        # Se combinar flujos de caja para la flota (teniendo en cuenta su respectiva cantidad)
        flujo_neto_comb = round(
            cantidad_ve * flujo_ve['Flujo Neto Descontado'].sum() +
            cantidad_conv * flujo_conv['Flujo Neto Descontado'].sum(), 2
        )
        
        ingreso_comb =  round(
            cantidad_ve * flujo_ve['Ingresos Descontado'].sum() +
            cantidad_conv * flujo_conv['Ingresos Descontado'].sum(), 2
        )

        # Se calcula  la inversión inicial combinada
        inversion_comb = round(
            cantidad_ve * auto_electrico.inversion_inicial() +
            cantidad_conv * auto_convencional.inversion_inicial(), 2
        )
        
        # Se calcular las métricas financieras combinadas
        roi_comb = flujo_neto_comb/ inversion_comb *100 / 7
        #roi_mensual_simple = roi_comb/12
        ir_comb = ingreso_comb / abs(inversion_comb)
        tir_comb = round(npf.irr(flujo_neto_comb_lista)*100,2)

        # Calculo del payback period combinado (considerando la proporción)
        flujo_anual_comb = [
            cantidad_ve * flujo_ve['Flujo Neto'].iloc[año] +
            cantidad_conv * flujo_conv['Flujo Neto'].iloc[año]
            for año in range(1, 6)
        ]
        flujo_acumulado = 0
        payback_comb = 0
        for año, flujo in enumerate(flujo_anual_comb, start=1):
            flujo_acumulado += flujo
            if flujo_acumulado >= abs(inversion_comb):
                payback_comb = año
                break
        
        # Se almacenan los resultados para el modelo actual
        resultados.append({
            'Brand': marca,
            'Model': modelo,
            'Efficiency Total (kWh/mile)' : round(eficiencia_total,4),
            'Inversión Inicial Total (USD)': round(inversion_comb,0),
            #'Relación Inversión/Eficiencia (USD.mile/kWh)': round (inversion_comb/eficiencia_total,2),
            'VNA (USD)': round(flujo_neto_comb,0),
            'TIR (%)':round(tir_comb,2),
            'ROI Anual (%)': round(roi_comb,2),
            #'ROI Mensual(%)': round(roi_mensual_simple,4),
            'IR (USD)': round(ir_comb,2),
            'Payback Period (Años)': payback_comb if payback_comb > 0 else "No recuperado en 7 años"
        })

    # Se convertierten los resultados a tipo DataFrame para una mejor visualización
    df_resultados = pd.DataFrame(resultados)
    df_resultados = df_resultados.sort_values(by="ROI Anual (%)", ascending=False).reset_index(drop=True) # Se ordena para tener el ranking 


    return df_resultados


Ejemplo de uso:

In [12]:
tasa_descuento = 0.10 # Primer Parámetro : Cantidad VE; Segundo Parámetro : Cantidad Autos Convencional
cantidad_ve = 1
cantidad_conv = 1
resultados_flota = calcular_metricas_flota(tasa_descuento, cantidad_ve, cantidad_conv)
resultados_flota.head(5) # head (5) para obtener el Ranking 

,Brand,Model,Efficiency Total (kWh/mile),Inversión Inicial Total (USD),VNA (USD),TIR (%),ROI Anual (%),IR (USD),Payback Period (Años)
0,SEAT,Mii Electric,0.6880,62394.0,26175.0,11.47,5.99,9.99,4
1,Smart,EQ fortwo coupe,0.6881,63728.0,24840.0,10.70,5.57,9.78,4
2,Volkswagen,e-Up!,0.6880,63764.0,24805.0,10.68,5.56,9.78,4
3,Smart,EQ forfour,0.6889,64410.0,24147.0,10.31,5.36,9.68,4
4,Skoda,CITIGOe iV,0.6880,67064.0,21505.0,8.88,4.58,9.30,4


## Calculo del impacto
### Impacto Incremental

En esta sección se calcula el impacto incremental de agregar un vehiculo convencional a la flota de taxis.

In [20]:
# Se simulan escenarios con x cantidad de VE y x cantidad de autos convencionales
# Para ello se crea un rango de vehículos eléctricos y convencionales
rango_ve = range(1, 11)  # De 1 a 10 VE
rango_conv = range(0, 11)  # De 0 a 10 Conv

# Se iniciliza una lista vacia para mas tarde agregar los valores
resultados = []

# Simulación de las combinaciones o escenarios
for ve in rango_ve:
    for conv in rango_conv:
        # Se calculan las métricas para la combinación del escenario actual
        resultados_flota = calcular_metricas_flota(tasa_descuento=0.10, cantidad_ve=ve, cantidad_conv=conv)
        
        # Se extraen los valores de las metricas 
        van = resultados_flota['VNA (USD)'].iloc[0]  # Para extraer la primera fila
        tir = resultados_flota['TIR (%)'].iloc[0] # Para extraer la primera fila
        roi = resultados_flota['ROI Anual (%)'].iloc[0] # Para extraer la primera fila
        payback = resultados_flota['Payback Period (Años)'].iloc[0] # Para extraer la primera fila
        ir = resultados_flota['IR (USD)'].iloc[0] # Para extraer la primera fila
        # Se almacenan los resultados en la lista antes inicilizada
        resultados.append({'VE': ve, 'Conv': conv, 'VAN': van, 'TIR': tir, 'ROI': roi, 'IR' : ir ,'Payback' : payback})

# Se convierte a formato DataFrame para mejor visualización.
df_resultados = pd.DataFrame(resultados)


In [21]:
# Aqui se agrupan por cada cantidad de VE para ver el comportamiento de los KPIs por cada vehiculo convencional incluido
df_resultados['Impacto_VAN'] = df_resultados.groupby('VE')['VAN'].diff() # Con diff saca al diferencia de la fila anterior
df_resultados['Impacto_TIR'] = df_resultados.groupby('VE')['TIR'].diff()
df_resultados['Impacto_ROI'] = df_resultados.groupby('VE')['ROI'].diff()
df_resultados['Impacto_Payback'] = df_resultados.groupby('VE')['Payback'].diff()
df_resultados['Impacto_IR'] = df_resultados.groupby('VE')['IR'].diff()

In [26]:
# Se obtiene el promedio del 'impacto'
impacto_promedio_van = df_resultados['Impacto_VAN'].mean()
impacto_promedio_tir = df_resultados['Impacto_TIR'].mean()
impacto_promedio_roi = df_resultados['Impacto_ROI'].mean()
impacto_promedio_payback = df_resultados['Impacto_Payback'].mean()
impacto_promedio_ir = df_resultados['Impacto_IR'].mean()


# Se imprime el valor del impacto incremental
print(f"Impacto promedio incremental de cada auto convencional incluido en VAN: {impacto_promedio_van:.2f}")
print(f"Impacto promedio incremental de cada auto convencional incluido en TIR: {impacto_promedio_tir:.2f}")
print(f"Impacto promedio incremental de cada auto convencional incluido en ROI: {impacto_promedio_roi:.2f}")
print(f"Impacto promedio incremental de cada auto convencional incluido en Payback: {impacto_promedio_payback:.2f}")
print(f"Impacto promedio incremental de cada auto convencional incluido en IR: {impacto_promedio_ir:.4f}")

Impacto promedio incremental de cada auto convencional incluido en VAN: 1291.40
Impacto promedio incremental de cada auto convencional incluido en TIR: -2.07
Impacto promedio incremental de cada auto convencional incluido en ROI: -1.19
Impacto promedio incremental de cada auto convencional incluido en Payback: 0.12
Impacto promedio incremental de cada auto convencional incluido en IR: -0.4733


### Impacto Relativo (Promedio General)
Esta sección compara un escenario combinado con uno exclusivo de vehículos eléctricos y refleja el impacto relativo. 

In [ ]:
ve_range = range(1,2) # Rango de simulaciones de cantidad de VE
conv_range = range(0,2) # Rango de simulaciones de cantidad de autos convencionales

resultados = []

# Simulaciones
for ve in ve_range:
    for conv in conv_range:
        resultados_flota = calcular_metricas_flota(0.1, ve, conv) # Obtención de métricas correspondiente a cantidad de autos (Ve y Conv)
        resultados_ROI = resultados_flota["ROI Anual (%)"].iloc[0]
        resultados_IR = resultados_flota["IR (USD)"].iloc[0]
        resultados_TIR = resultados_flota["TIR (%)"].iloc[0]
        resultados_Payback = resultados_flota["Payback Period (Años)"].iloc[0]
        resultados_VNA = resultados_flota["VNA (USD)"].iloc[0]  
        resultados.append({'VE' : ve, 'Conv' : conv, 'ROI' : resultados_ROI, 'IR' : resultados_IR, 'TIR' : resultados_TIR, 
         'Payback' : resultados_Payback, 'VNA' : resultados_VNA})
        
resultados = pd.DataFrame(resultados)

# Calculo de impactos relativos

impacto_ROI = round(((resultados.iloc[0,2] - resultados.iloc[1,2]) / resultados.iloc[1,2]) * 100,2) 
impacto_IR = round(((resultados.iloc[0,3] - resultados.iloc[1,3]) / resultados.iloc[1,3]) * 100,2)  
impacto_TIR = round(((resultados.iloc[0,4] - resultados.iloc[1,4]) / resultados.iloc[1,4]) * 100,2)
impacto_payback = round(((resultados.iloc[0,5] - resultados.iloc[1,5]) / resultados.iloc[1,5]) * 100,2)
impacto_vna = round(((resultados.iloc[0,6] - resultados.iloc[1,6]) / resultados.iloc[1,6]) * 100,2)

print(f'El impacto relativo de una flota combinada frente a una exclusiva en el indicador ROI es: {impacto_ROI:.2f}%')
print(f'El impacto relativo de una flota combinada frente a una exclusiva en el indicador IR es: {impacto_IR:.2f}%')
print(f'El impacto relativo de una flota combinada frente a una exclusiva en el indicador TIR es: {impacto_TIR:.2f}%')
print(f'El impacto relativo de una flota combinada frente a una exclusiva en el indicador Payback es: {impacto_payback:.2f}%')
print(f'El impacto relativo de una flota combinada frente a una exclusiva en el indicador VNA es: {impacto_vna:.2f}%')

#resultados.head(10)

El impacto relativo de una flota combinada frente a una exclusiva en el indicador ROI es: 164.61%
El impacto relativo de una flota combinada frente a una exclusiva en el indicador IR es: 39.14%
El impacto relativo de una flota combinada frente a una exclusiva en el indicador TIR es: 146.56%
El impacto relativo de una flota combinada frente a una exclusiva en el indicador Payback es: -25.00%
El impacto relativo de una flota combinada frente a una exclusiva en el indicador VNA es: -4.94%
